In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import torch
import torch.nn as nn
import torch
from torch import nn
from sklearn.model_selection import KFold
from itertools import product
from cup_helpers import SEED
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from cup_helpers import CV
from torch.optim.lr_scheduler import StepLR
from sklearn.discriminant_analysis import StandardScaler
import time




In [2]:
INPUT_SIZE = 10
OUTPUT_SIZE = 3

Create reproducible results

In [3]:
torch.manual_seed(SEED)
np.random.seed(SEED)
gen = torch.Generator().manual_seed(SEED)
SEEDS = list(range(40,46))



In [4]:
# Datasets Path
TR_PATH = "./data/ML-CUP23-TR.csv"
TS_PATH = "./data/ML-CUP23-TS.csv"

In [5]:

def plot_graph(train_losses,losses,epochs,title):
    num_epochs = list(range(0, epochs))  
    print(len(num_epochs))
    print((train_losses))
    print((losses))
    # Plotting
    plt.plot(num_epochs, train_losses, label=' Training Loss ')
    plt.plot(num_epochs, losses, label=title+' Loss')

    plt.title('Training and '+title+' Losses Across Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [6]:
class Net(nn.Module):
  def __init__(self, input_size,layers_size, activation_function="relu"):
    super().__init__()
    self.layers = nn.ModuleList()
    self.activation_function = activation_function
    for layer_size in layers_size:
      self.layers.append(nn.Linear(input_size,layer_size))
      input_size = layer_size

  def forward(self, x):
    for i, layer in enumerate(self.layers):
            if i != (len(self.layers) - 1):
              # Apply the layer and a ReLU activation
              if self.activation_function == "relu":   
                x = torch.relu((layer(x)))
              elif self.activation_function == "tanh":
                x = torch.tanh((layer(x)))
            else:
              x = layer(x)
    return x


def reset_weights(net):
  for param in net.parameters():
    torch.nn.init.uniform_(param, a=-0.7, b=0.7)

In [7]:


def read_ds(path,split):
  """
  parse CSV data set and
  returns a tuple (input, target)
  """
  data = pd.read_csv(path, dtype=object, delimiter=",", header=None, skiprows=1, names=["id", "INPUT_0", "INPUT_1", "INPUT_2", "INPUT_3", "INPUT_4", "INPUT_5", "INPUT_6", "INPUT_7", "INPUT_8", "INPUT_9", "TARGET_x", "TARGET_y", "TARGET_z"])
  y = data.drop(["id","INPUT_0", "INPUT_1", "INPUT_2", "INPUT_3", "INPUT_4", "INPUT_5", "INPUT_6", "INPUT_7", "INPUT_8", "INPUT_9"], axis=1)
  X = data.drop(["id","TARGET_x", "TARGET_y", "TARGET_z"], axis=1).astype(float).to_numpy()

  y = y.astype(float).to_numpy()
  
  if split == True:
    # Train/internal test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED)
    return (np.concatenate((y_train, X_train), axis=1),np.concatenate((y_test, X_test), axis=1))
  else:
    return (np.concatenate((y, X), axis=1))

In [8]:
dataset_train,dataset_internal_test = read_ds(TR_PATH,True)

In [9]:
dataset_train[0]

array([-2.1704154e+00, -7.3649310e+01,  9.4287815e+00,  8.6179230e-02,
       -9.8935485e-01,  9.1387004e-01,  5.2771106e-02,  9.8712380e-01,
        9.9955016e-01,  8.5003810e-01, -9.1003710e-01, -7.2887754e-01,
        4.4358963e-01])

In [10]:
class EarlyStopper:
  def __init__(self, epochs_to_wait=1, min_delta=0):
    self.min_training_loss = np.inf
    self.epochs_to_wait = epochs_to_wait
    self.min_delta = min_delta
    self.counter = 0

  def check_early_stop(self, training_loss):
    if training_loss > (self.min_training_loss - self.min_delta):
      self.counter +=1
      if self.counter >= self.epochs_to_wait:
        return True
    else: 
      self.counter = 0
    if training_loss < self.min_training_loss:
      self.min_training_loss = training_loss
    
    return False

In [11]:
def Padding(validation_losses_fold,train_losses_fold):
    max_epochs = max(map(len, validation_losses_fold))

    for validation_loss_arr in validation_losses_fold:
        while len(validation_loss_arr) < max_epochs:
            validation_loss_arr.append(validation_loss_arr[-1])

    for train_loss_arr in train_losses_fold:
      while len(train_loss_arr) < max_epochs:
            train_loss_arr.append(train_loss_arr[-1])

    print(len(validation_losses_fold))
    return validation_losses_fold,train_losses_fold

In [12]:
def Mean(validation_avg_loss_fold,train_losses_fold,n_folds):
    max_epochs = max(map(len, validation_avg_loss_fold))
    validation_avg_loss = []
    train_avg_loss = []
    
    for i in range(0,max_epochs):
        temp_loss = 0
        for j in range(0,len(validation_avg_loss_fold)):
            temp_loss += validation_avg_loss_fold[j][i]
        validation_avg_loss.append(temp_loss/n_folds)

    for i in range(0,max_epochs):
        temp_loss = 0
        for j in range(0,len(train_losses_fold)):
            temp_loss += train_losses_fold[j][i]
        train_avg_loss.append(temp_loss/n_folds)
    
    return validation_avg_loss,train_avg_loss

In [13]:
def fit_model(input_size,layers_size,learning_rate,momentum,weight_decay,opt,epochs,trainloader,loss_function,testloader, activation_function):
    # Init the neural network
    network = Net(input_size, layers_size, activation_function)
    network.apply(reset_weights) #reset weights with random initialization

    optimizer = opt(network.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

    train_MEE_folds = []
    test_MEE_folds = []

    # Run the training loop for defined number of epochs
    for epoch in range(0, epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss and accuracy value for train
      train_loss = 0.0
      train_MEE = 0.0

      # Set current loss and accuracy value for test
      test_loss = 0.0
      test_MEE = 0.0
     

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        # Get inputs
        inputs = data[:, OUTPUT_SIZE:].to(torch.float32)
        targets = data[:, :OUTPUT_SIZE].to(torch.float32)

        # Zero the gradients
        optimizer.zero_grad() 

        
        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()

        
        # Print statistics
        train_loss += loss.item() * inputs.size(0)
        #caluclate MEE for this batch pow(2), sum(axis = 1), sqrt(), sum()
        train_MEE += float((outputs-targets).pow(2).sum(1).sqrt().sum())

      # Print loss values
      epoch_train_loss = train_loss / len(trainloader.sampler)    
      print("TRAIN ERROR MSE:",epoch_train_loss)

      train_MEE = train_MEE / len(trainloader.sampler) 
      train_MEE_folds.append(train_MEE)
      print("TRAIN ERROR MEE:",train_MEE_folds) 
      
      with torch.no_grad():
        # Iterate over the testing data and generate predictions
        for i, data in enumerate(testloader, 0):

          inputs = data[:, OUTPUT_SIZE:].to(torch.float32)
          targets = data[:, :OUTPUT_SIZE].to(torch.float32)
        
          outputs = network(inputs)

          loss = loss_function(outputs, targets)
          
          test_loss += loss.item() * inputs.size(0)
          test_MEE += float((outputs-targets).pow(2).sum(1).sqrt().sum())

        epoch_test_loss = test_loss / len(testloader.sampler)    
        print("TEST ERROR MSE:",epoch_test_loss)
     
        test_MEE = test_MEE / len(testloader.sampler) 
        test_MEE_folds.append(test_MEE)
        print("TEST ERROR MEE:",test_MEE_folds)

    plot_graph(train_MEE_folds,test_MEE_folds,epoch+1,"test")
    return network,train_MEE_folds[-1],test_MEE_folds[-1]


In [14]:
def k_fold_model(input_size,layers_size,learning_rate,momentum,weight_decay,opt,epochs,trainloader,loss_function,validationloader,lr_decay,lr_step_size, activation_function):
    
    # Init the neural network 
    network = Net(input_size, layers_size, activation_function)
    network.apply(reset_weights) #reset weights with random initialization
    
    optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    #scheduler = StepLR(optimizer, step_size=lr_step_size, gamma=lr_decay)

    early_stopper = EarlyStopper(epochs_to_wait=30, min_delta=1e-5)

    train_MEE_folds = []
    validation_MEE_folds = []
    # Run the training loop for defined number of epochs
    for epoch in range(0, epochs):
      
      # Set current loss value
      train_loss = 0.0

      #Set current MEE metric
      train_MEE = 0.0
      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        # Get inputs
        inputs = data[:, OUTPUT_SIZE:].to(torch.float32)
        targets = data[:, :OUTPUT_SIZE].to(torch.float32)
        # Zero the gradients
        optimizer.zero_grad() 
        
        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()

        # Print statistics
        train_loss += loss.item() * inputs.size(0)

        #caluclate MEE for this batch pow(2), sum(axis = 1), sqrt(), sum()
        train_MEE += float((outputs-targets).pow(2).sum(1).sqrt().sum())

      avg_train_loss = train_loss / len(trainloader.sampler)    

      train_MEE = train_MEE / len(trainloader.sampler) 
      train_MEE_folds.append(train_MEE)
      


      # Evaluationfor this fold
      valid_loss = 0.0 
      val_MEE = 0.0

      with torch.no_grad():
        # Iterate over the validation data and generate predictions
        for i, data in enumerate(validationloader, 0):

          # Get inputs
          inputs = data[:, OUTPUT_SIZE:].to(torch.float32)
          targets = data[:, :OUTPUT_SIZE].to(torch.float32)
          
          # Generate outputs
          outputs = network(inputs)

          loss = loss_function(outputs, targets)

          # Calculate loss
          valid_loss += loss.item() * inputs.size(0)
          val_MEE += float((outputs-targets).pow(2).sum(1).sqrt().sum())

        avg_valid_loss = valid_loss / len(validationloader.sampler) #used to find the best parameters of the model
        val_MEE = val_MEE / len(validationloader.sampler)
        validation_MEE_folds.append(val_MEE)
        
        # Early stopping
        if early_stopper.check_early_stop(avg_valid_loss):
          print("Early stopping:", epoch)
          break


    return avg_valid_loss,avg_train_loss,validation_MEE_folds,train_MEE_folds


In [15]:
def execute_folds(kfold,dataset,batch_size,input_size, layers_size, learning_rate, epochs,
    loss_function, momentum, opt, weight_decay,lr_decay,lr_step_size, activation_function):
    
    validation_avg_mee_fold = 0
    train_avg_mee_fold = 0
    validation_MEE_fold = []
    train_MEE_fold = []
    num_iterations = 0
    current_config = {
        'input_size': input_size,
        'layers_size': layers_size,
        'learning_rate': learning_rate,
        'epochs': epochs,
        'momentum': momentum,
        'opt': opt,
        'weight_decay': weight_decay,
        "batch_size":batch_size,
        "loss_function":loss_function,
        "lr_decay":lr_decay,
        "lr_step_size":lr_step_size,
        "activation_function": activation_function,
    }
    print(current_config)
    for fold, (train_ids, val_ids) in enumerate(kfold.split(np.zeros(len(dataset)),dataset[:, 0])):
        
        
        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids, gen) 
        validation_subsampler = torch.utils.data.SubsetRandomSampler(val_ids, gen) 

        # Define data loaders for training and testing data in this fold
        
        trainloader = torch.utils.data.DataLoader(
                        dataset, 
                        batch_size=batch_size, sampler=train_subsampler)
        validationloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=validation_subsampler)    
        

        validation_loss,train_loss,validation_MEE,train_MEE = k_fold_model(learning_rate=learning_rate,epochs=epochs,layers_size=layers_size,input_size=input_size,loss_function=loss_function,momentum=momentum
                                                    ,opt=opt,trainloader=trainloader,weight_decay=weight_decay,validationloader=validationloader,lr_decay=lr_decay,lr_step_size=lr_step_size, activation_function=activation_function)   
        validation_avg_mee_fold  += validation_loss
        train_avg_mee_fold += train_loss
        validation_MEE_fold.append(validation_MEE)
        train_MEE_fold.append(train_MEE)

        num_iterations += 1


    #validation average over all folds
    validation_avg_mee_fold /= num_iterations
    train_avg_mee_fold /= num_iterations

    
    validation_MEE_fold,train_MEE_fold = Padding(validation_MEE_fold,train_MEE_fold)

    validation_MEE_mean, train_MEE_mean = Mean(validation_MEE_fold, train_MEE_fold,n_folds=num_iterations)


    plot_graph(train_MEE_mean,validation_MEE_mean,len(train_MEE_mean),"validation")


    return (validation_avg_mee_fold,train_avg_mee_fold,current_config)

function that executes the folds for each combination of parameters

In [16]:


def dogridsearch(dataset_train_part,params_grid,input_size,seeds):
    dataset = dataset_train_part
    # Set fixed random number seed
    loss_function = nn.MSELoss()

    # Define the K-fold Cross Validator
    kfold = CV 
    
    # K-fold Cross Validation model evaluation
    best_params = None

    actual_it = 0
    total_iterations = len(params_grid["epochs"]) * len(params_grid["optimizer"]) * len(params_grid["layers_size"]) \
          * len(params_grid["learning_rate"]) * len(params_grid["batch_size"]) * len(params_grid["weight_decay"]) * len(params_grid["momentum"]) * len(params_grid["lr_decay"]) * len(params_grid["lr_step_size"] * len(params_grid["activation_function"]))


    configurations = []

    for epochs, opt, layers_size, learning_rate, batch_size, weight_decay, momentum,lr_decay,lr_step_size, activation_function in product(params_grid["epochs"],params_grid["optimizer"], 
                                                                                               params_grid["layers_size"], params_grid["learning_rate"], 
                                                                                               params_grid["batch_size"], params_grid["weight_decay"], 
                                                                                               params_grid["momentum"],params_grid["lr_decay"],params_grid["lr_step_size"], params_grid["activation_function"]):
        #print the actual percentage of the grid search
        print(f'Actual iter {(actual_it/total_iterations)*100}%')
        for seed in seeds:
            print("Working with seed:",seed)
            torch.manual_seed(seed)

            (validation_avg_loss_fold,train_avg_loss_fold,current_config) = execute_folds(kfold=kfold,dataset=dataset,learning_rate=learning_rate,epochs=epochs,layers_size=layers_size,input_size=input_size,loss_function=loss_function,momentum=momentum
                                                            ,opt=opt,weight_decay=weight_decay,batch_size=batch_size,lr_decay =lr_decay,lr_step_size=lr_step_size, activation_function=activation_function)


            configurations.append((validation_avg_loss_fold, train_avg_loss_fold,current_config))
        
        actual_it += 1

    val_mee = []
    train_mee = []
    #best 
    for conf_val in configurations:
        val_mee.append(conf_val[0])
        train_mee.append(conf_val[1])
        if best_params is None or conf_val[0] < best_params[0]:
                current_config = conf_val[2]
                best_params = (conf_val[0],conf_val[1]
                ,current_config['learning_rate'], current_config['epochs'],current_config["loss_function"],current_config['layers_size'],current_config['momentum'],current_config['opt'],
                current_config['weight_decay'],
                current_config['batch_size'],current_config["lr_decay"],current_config["lr_step_size"], current_config["activation_function"])
    print("TRAIN MEAN MEE",np.mean(train_mee))
    print("TRAIN STD",np.std(train_mee))
    print("VALIDATION MEAN MEE",np.mean(val_mee))
    print("VALIDATION STD",np.std(val_mee))
    return best_params

test the created model and plot training/test error

In [17]:
def train_test_model(dataset_train_part,dataset_test_part,best_params,seeds):
    train_subsampler = torch.utils.data.SubsetRandomSampler(range(len(dataset_train_part)), gen)

    trainloader = torch.utils.data.DataLoader(
                        dataset_train_part, 
                        batch_size=best_params[9], sampler=train_subsampler)

    test_subsampler =  torch.utils.data.SubsetRandomSampler(range(len(dataset_test_part)), gen)
    testloader = torch.utils.data.DataLoader(
                        dataset_test_part, 
                        batch_size=best_params[9], sampler=test_subsampler)
    train_mee_seed = []
    test_mee_seed = []
    for seed in seeds:
        # Start the timer
        start = time.time()
        print("Working with seed:",seed)
        torch.manual_seed(seed)
        best_net,train_mee,test_mee = fit_model(learning_rate=best_params[2],epochs=best_params[3],layers_size=best_params[5],input_size=INPUT_SIZE,loss_function=best_params[4],
                        momentum=best_params[6],opt=best_params[7],trainloader=trainloader,weight_decay=best_params[8],activation_function=best_params[12],testloader=testloader)   
        train_mee_seed.append(train_mee)
        test_mee_seed.append(test_mee)
        end = time.time()
        print("Refit Time: {:.2f} seconds".format(end - start))
    
    print("TRAIN MEAN MEE",np.mean(train_mee_seed))
    print("TRAIN STD",np.std(train_mee_seed))
    print("TEST MEAN MEE",np.mean(test_mee_seed))
    print("TEST STD",np.std(test_mee_seed))

    return best_net

In [ ]:
params_grid = {
    "layers_size": [[128, 64, 32, OUTPUT_SIZE]], #we tried all possibile combinations of 2 layers [2,4,8,16,32,64,128,256,512] and
     # (due to computation limits) combinations of 3 layers decreasing, increasing order, and with the central layer bigger or smaller than the others
    "learning_rate": [0.001], #0.0005, 0.001, 0.005, 0.01, 0.1, 0.5, 0.7, 0.8
    "batch_size": [1000], #16,32,64,128,256,512, 1000 corresponds to batch method, no mini batch
    "weight_decay": [0.1], #0.001, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5
    "momentum": [ 0.5], #0.01, 0.1, 0.5, 0.7, 0.8
    "epochs":[1200], 
    "optimizer":[torch.optim.SGD],
    "lr_decay":[0], #0.1, 0.01, 0.5 we tried first with lr_decay and then we removed it
    "lr_step_size":[0], #0,32,64
    "activation_function":["relu"] #relu, tanh
}




dataset = dataset_train

best_params = dogridsearch(dataset_train_part=dataset,params_grid=params_grid,input_size=INPUT_SIZE,seeds = SEEDS)


In [ ]:
#Best parameters found
print(f"Best hidden size: {best_params[5]} \nBest learning rate: {best_params[2]} \nBest batch size: {best_params[9]} \nBest weight decay: {best_params[8]} \nBest momentum: {best_params[6]} \nBest epochs: {best_params[3]} \nBest optimizer: {best_params[7]} \nBest lr_decay: {best_params[10]} \nBest lr_step_size: {best_params[11]}\nBest activation_function: {best_params[12]}")

print(best_params)

In [ ]:
best_net = train_test_model(dataset_train_part=dataset_train,dataset_test_part=dataset_internal_test,best_params=best_params,seeds = SEEDS)  